In [2]:
pip install gensim


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gensim

In [4]:
import pandas as pd
import numpy as np

In [5]:
temp_df=pd.read_csv("IMDB Dataset.csv")

In [6]:
temp_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df = temp_df.iloc[:10000]
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.drop_duplicates(inplace=True)

/var/folders/hn/_nb8yt5s35z4zn8wghrqt2bw0000gn/T/ipykernel_2035/3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [9]:
import re
def remove_tag(raw_text):
    cleaned_text=re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [10]:
df['review']=df['review'].apply(remove_tag)

/var/folders/hn/_nb8yt5s35z4zn8wghrqt2bw0000gn/T/ipykernel_2035/3179549256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_tag)


In [11]:
df['review'],df['review'].apply(lambda x:x.lower())

(0       One of the other reviewers has mentioned that ...
 1       A wonderful little production. The filming tec...
 2       I thought this was a wonderful way to spend ti...
 3       Basically there's a family where a little boy ...
 4       Petter Mattei's "Love in the Time of Money" is...
                               ...                        
 9995    Fun, entertaining movie about WWII German spy ...
 9996    Give me a break. How can anyone say that this ...
 9997    This movie is a bad movie. But after watching ...
 9998    This is a movie that was probably made to ente...
 9999    Smashing film about film-making. Shows the int...
 Name: review, Length: 9983, dtype: object,
 0       one of the other reviewers has mentioned that ...
 1       a wonderful little production. the filming tec...
 2       i thought this was a wonderful way to spend ti...
 3       basically there's a family where a little boy ...
 4       petter mattei's "love in the time of money" is...
            

In [12]:
from nltk.corpus import stopwords

sw_list=stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [word for word in x.split() if word not in sw_list]).apply(lambda x: " ".join(x))

/var/folders/hn/_nb8yt5s35z4zn8wghrqt2bw0000gn/T/ipykernel_2035/1621938133.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [word for word in x.split() if word not in sw_list]).apply(lambda x: " ".join(x))


In [13]:
df['review']

0       One reviewers mentioned watching 1 Oz episode ...
1       A wonderful little production. The filming tec...
2       I thought wonderful way spend time hot summer ...
3       Basically there's family little boy (Jake) thi...
4       Petter Mattei's "Love Time Money" visually stu...
                              ...                        
9995    Fun, entertaining movie WWII German spy (Julie...
9996    Give break. How anyone say "good hockey movie"...
9997    This movie bad movie. But watching endless ser...
9998    This movie probably made entertain middle scho...
9999    Smashing film film-making. Shows intense stran...
Name: review, Length: 9983, dtype: object

In [14]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [15]:
story =[]
for doc in df['review']:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [16]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [17]:
model.build_vocab(story)

In [18]:
model.train(story,total_examples=model.corpus_count, epochs=model.epochs)

(6100143, 6529530)

In [19]:
len(model.wv.index_to_key)

31845

In [25]:
import numpy as np

def documents_vector(doc, model):
    # Remove out of vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    
    # If the document is empty after filtering, return a vector of zeros
    if len(doc) == 0:
        return np.zeros(model.vector_size)
    
    # Compute the mean of word vectors
    return np.mean([model.wv[word] for word in doc], axis=0)

In [27]:
document_vector = documents_vector(df['review'].values[0], model)

In [29]:
pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
import tqdm

In [31]:
from tqdm import tqdm

In [41]:
x = []
for doc in tqdm(df['review'].values):
    x.append(documents_vector(doc, model))  # Pass 'model' explicitly

100%|███████████████████████████████████████| 9983/9983 [02:56<00:00, 56.61it/s]


In [48]:
x=np.array(x)

In [49]:
x[0]

array([-0.07499494,  0.43006307,  0.11793921, -0.0216762 ,  0.11981379,
       -0.60299063,  0.08494354,  0.73537743, -0.21906149, -0.14651863,
       -0.28664136, -0.66350734, -0.15857054,  0.20437174, -0.06645857,
       -0.41858408,  0.06211361, -0.21544848,  0.09672651, -0.70157343,
        0.03134253,  0.04363453,  0.24727838,  0.04447424,  0.11581209,
        0.0077533 , -0.17821278, -0.04735409, -0.19873607, -0.07800749,
        0.43329984,  0.05451129,  0.23329309, -0.25770432, -0.10635544,
        0.22842276,  0.03528594, -0.38610682, -0.15679984, -0.98052794,
        0.06670064, -0.40413558, -0.12079145, -0.06620859,  0.06818949,
       -0.08710723, -0.28680915,  0.02661522,  0.13826589,  0.2660464 ,
        0.29157364, -0.21037732, -0.31156734,  0.01138517, -0.12374344,
        0.07922688,  0.1899503 , -0.25366336, -0.41704538,  0.12832287,
        0.22801493,  0.27193838, -0.07935191,  0.00984083, -0.46582943,
        0.04355253,  0.10090164,  0.31041694, -0.59621263,  0.33

In [50]:
from sklearn.preprocessing import LabelEncoder

In [51]:
encoder = LabelEncoder()

y=encoder.fit_transform(df['sentiment'])

In [52]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [61]:
from sklearn.model_selection import train_test_split

In [63]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [70]:
rf=RandomForestClassifier()
Classifier = RandomForestClassifier(n_estimators=500,random_state=42)

In [72]:
Classifier.fit(x_train,y_train)
y_pred = Classifier.predict(x_test)

In [74]:
accuracy = accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy:2f}")

Accuracy: 0.778167
